In [33]:
# A_Aug_Swap_Flag_Class_over_Flag_Class_with_BG_Fill.jpynb
"""
# Include in set for dissertation 

Script for Merging Flag Classes in Images with Background Filling and Overlay Replacement
This script processes a dataset of images and performs the following steps:
1. - Identifies the bounding box of a target class in the target image.
   - Fills the bounding box with a background color sampled from its surroundings.
   - Applies optional padding to ensure smooth blending.

2. - Extracts a region of interest (ROI) of a specified class from a source image.
    - Aligns the ROI with the target bounding box in the target image.
   - Blends the extracted ROI with the target image using a binary mask and Gaussian smoothing.
   - Supports custom alignment options ("top-left" or "top-right") and padding - for flagpole on left or right.

3. - Creates a grid of images showing pipeline stages for debugging and demonstration:
        - Source image
        - Target image
        - Target image with background filling
        - Final blended image with overlay

4.  - Updates YOLO- annotations for the target image to reflect the new overlay.

Global Variables:
- `merge_class_from`: Path to the folder containing source images.
- `merge_class_into`: Path to the folder containing target images.
- `merge_fldr`: Path to save processed images and debugging outputs.
- `class_to_take`: Source class ID to be overlaid on the target class.
- `class_to_replace`: Target class ID to be replaced.
- `scale_factor`: Scaling factor for resizing the overlay.
- `x_offset`: Horizontal offset for overlay alignment.
- `edge_blend_width`: Width of the blending edge.
- `threshold`: Intensity threshold for binary mask creation.
- `align`: Alignment for overlay positioning ("top-left" or "top-right").
- `padding`: Padding values for bounding box filling (top, right, bottom, left).

Key Functions:
- `create_merged_filename`: Generates filenames for the merged images and annotations.
- `visualize_pipeline_steps`: Saves a grid visualization of the pipeline stages.
- `fill_target_bbox_with_background`: Fills the bounding box of the target class with sampled background colors.
- `overlay_class_region`: Overlays the source ROI onto the target bounding box with alignment, padding, and blending.

How to run 
 Check that `merge_class_from`, `merge_class_into`, and `merge_fldr` directories exist and contain the appropriate images and annotations. 
 Then run: process_switch_class(
        src_folder=merge_class_from,
        tgt_folder=merge_class_into,
        merge_folder=merge_fldr,
        target_class=class_to_replace,
        source_class_id=class_to_take,
        scale_factor=scale_factor,
        x_offset=x_offset,
        flip_src=False,
        flip_tgt=False
    )


### """

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt


def create_merged_filename(file_name, merge_from_job, source_class_id, merge_to_job, target_class):
    """
    Create a new filename for the merged image or annotation based on user specifications.

    Args:
        file_name (str): The original filename.
        merge_from_job (str): Identifier for the source job.
        source_class_id (int): The class ID from the source.
        merge_to_job (str): Identifier for the target job.
        target_class (int): The target class ID being replaced.

    Returns:
        str: A new filename that follows the specified naming convention.
    """
    base_name, ext = os.path.splitext(file_name)
    new_name = (f"Merge_{merge_from_job}_CLS_{source_class_id}_"
                f"into_{merge_to_job}_CLS_{target_class}_{base_name}{ext}")
    return new_name


def visualize_pipeline_steps(source_image, target_image, background_filled, final_image, file_name):
    #, , background_filled, final_image, output_folder, file_name):

    """
    Save a visualization of the pipeline steps as a grid.
    
    #Args: This didnt work when passing target images with the bounding box 
    #    target_image (ndarray): Original target image with the bounding box.
    #    background_filled (ndarray): Target image with bounding box filled with background color.
    #    resized_source (ndarray): Resized source region.
    #    final_image (ndarray): Final output image with overlay and blending.
    #    file_name (str): Name of the output visualization file.
    
    Args: Updating to read them from disk 
        source_path (str): Path to the source image file.
        target_path (str): Path to the target image file.
        filled_path (str): Path to the background-filled image file.
        final_path (str): Path to the final blended image file.
        file_name (str): Name of the output visualization file.
    """
    # Read images from paths instead 
    source_image = cv2.imread(os.path.join(merge_fldr,source_image))
    target_image = cv2.imread(os.path.join(merge_fldr, target_image))
    background_filled = cv2.imread(os.path.join(merge_fldr, background_filled))
    final_image = cv2.imread(os.path.join(merge_fldr, final_image))
    
    #target_image = cv2.resize(target_image, (source_image.shape[1], source_image.shape[0]))
    #background_filled = cv2.resize(background_filled, (source_image.shape[1], source_image.shape[0]))
    #final_image = cv2.resize(final_image, (source_image.shape[1], source_image.shape[0]))
    
    images = [source_image, target_image, background_filled, final_image]
    titles = ["1. Brightenned source image from which to extract flag", "2. Target image on which to overlay flag", "3. Background filled target", "4. Final blended output with enlarged flag swapped in"] 
    plt.figure(figsize=(12, 8))
    for i, (img, title) in enumerate(zip(images, titles)):
        plt.subplot(2, 2, i + 1)
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.axis("off")
    output_path = os.path.join(merge_fldr, f"F_pipeline_steps_{file_name}")
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()
    print(f"Visualization saved at: {output_path}")


def fill_target_bbox_with_background(tgt_image, tgt_annotations, target_class, padding, x_offset):
    """
    Fill the bounding box for the target class with the background color sampled from surrounding areas,
    applying directional padding.

    Args:
        tgt_image (ndarray): The target image.
        tgt_annotations (list): List of annotations (YOLO format).
        target_class (int): The class ID of the target bounding box.
        padding (tuple): Padding for (top, right, bottom, left) directions.
        x_offset (int): Horizontal offset applied to the bounding box.

    Returns:
        tgt_image (ndarray): The updated image with the target bounding box filled.
    """
    top_pad, right_pad, bottom_pad, left_pad = padding
    image_height, image_width = tgt_image.shape[:2]
    target_bbox = None

    for annotation in tgt_annotations:
        class_id, x_center, y_center, width, height = annotation
        if int(class_id) == target_class:
            # Apply x_offset to background fill or the overlay not both

            #x_min = int((x_center - width / 2) * image_width) + x_offset
            #x_max = int((x_center + width / 2) * image_width) + x_offset

            x_min = int((x_center - width / 2) * image_width)  #+ x_offset
            y_min = int((y_center - height / 2) * image_height)
            x_max = int((x_center + width / 2) * image_width)  #+ x_offset
            y_max = int((y_center + height / 2) * image_height)

            # Ensure coordinates are within bounds
            x_min = max(0, x_min - left_pad)
            y_min = max(0, y_min - top_pad)
            x_max = min(image_width, x_max + right_pad)
            y_max = min(image_height, y_max + bottom_pad)

            # Sample regions outside the bounding box
            top = tgt_image[max(0, y_min - top_pad):y_min, x_min:x_max]
            bottom = tgt_image[y_max:min(image_height, y_max + bottom_pad), x_min:x_max]
            left = tgt_image[y_min:y_max, max(0, x_min - left_pad):x_min]
            right = tgt_image[y_min:y_max, x_max:min(image_width, x_max + right_pad)]
            
            # Combine the mean colors from sampled regions
            sampled_colors = []
            if top.size > 0:
                sampled_colors.append(top.mean(axis=(0, 1)))
            if bottom.size > 0:
                sampled_colors.append(bottom.mean(axis=(0, 1)))
            if left.size > 0:
                sampled_colors.append(left.mean(axis=(0, 1)))
            if right.size > 0:
                sampled_colors.append(right.mean(axis=(0, 1)))

            if not sampled_colors:
                print("Sampling failed for bounding box. Skipping background filling.")
                continue

            # Calculate the average color from all sampled regions
            background_color = np.mean(sampled_colors, axis=0).astype(np.uint8)

            # Fill the bounding box with the computed background color
            tgt_image[y_min:y_max, x_min:x_max] = background_color
            target_bbox = (x_min, y_min, x_max, y_max)
            break

    if target_bbox is None:
        print("No bounding box found for target class. Skipping background filling.")

    return tgt_image


def overlay_class_region(
    file_name, src_image, src_annotations, source_class_id, tgt_image, tgt_annotations, 
    target_class, scale_factor, x_offset, edge_blend_width, 
    threshold, padding, align, visualize_once=True, debug_folder=None):
    """
    Overlay the source class region onto the target bounding box with alignment and optional padding.

    Args:
        src_image (ndarray): The source image.
        src_annotations (list): List of annotations for the source image.
        source_class_id (int): The class ID to extract from the source image.
        tgt_image (ndarray): The target image.
        tgt_annotations (list): List of annotations for the target image.
        target_class (int): The target class ID to replace.
        scale_factor (float): Scaling factor for resizing the extracted region.
        x_offset (int): Horizontal offset for overlay placement.
        edge_blend_width (int): Width of the edges to apply blending.
        threshold (int): Intensity threshold for binary mask creation.
        debug_folder (str): Folder path to save debugging visualizations. If None, debugging is skipped.
        align (str): Alignment option for the overlay ("top-left" or "top-right").
        padding (tuple): Padding to apply (top, right, bottom, left).
        visualize_once (bool): If True, visualizations will only occur for the first file processed.

    Returns:
        tuple: Updated target image, new annotation for the overlaid class, and updated annotations.
    """
    image_height, image_width = src_image.shape[:2]
    src_roi = None
    new_annotation = None
    # Flag to track if it's the first file being processed
    global visualized
    if 'visualized' not in globals():
        visualized = False
        
    # Extract the region of interest (ROI) for the source class
    for annotation in src_annotations:
        class_id, x_center, y_center, width, height = annotation
        if int(class_id) == source_class_id:
            box_width = int(width * image_width)
            box_height = int(height * image_height)
            x_center_px = int(x_center * image_width)
            y_center_px = int(y_center * image_height)

            x_min = max(0, x_center_px - box_width // 2)
            y_min = max(0, y_center_px - box_height // 2)
            x_max = min(image_width, x_center_px + box_width // 2)
            y_max = min(image_height, y_center_px + box_height // 2)

            src_roi = src_image[y_min:y_max, x_min:x_max]
            break

    if src_roi is None:
        print("No matching class found in the source image.")
        return tgt_image, new_annotation

    for annotation in tgt_annotations:
        
        class_id, x_center, y_center, width, height = annotation
        if int(class_id) == target_class:
            # Calculate the center and size of the target bounding box
            tgt_x_center = int(x_center * tgt_image.shape[1])  
            tgt_y_center = int(y_center * tgt_image.shape[0])
            tgt_width = int(width * tgt_image.shape[1] * scale_factor) 
            tgt_height = int(height * tgt_image.shape[0] * scale_factor)

            # Align the overlay based on the alignment option
            if align == "top-left":
                tgt_x_min = max(0, tgt_x_center - tgt_width // 2 + x_offset)
                tgt_y_min = max(0, tgt_y_center - tgt_height // 2)
            elif align == "top-right":
                tgt_x_min = max(0, tgt_x_center - tgt_width // 2 + x_offset)
                tgt_y_min = max(0, tgt_y_center - tgt_height // 2)
            else:
                raise ValueError("Invalid alignment option. Choose 'top-left' or 'top-right'.")

            # Apply padding
            top_pad, right_pad, bottom_pad, left_pad = padding
            tgt_x_min = max(0, tgt_x_min - left_pad)
            tgt_y_min = max(0, tgt_y_min - top_pad)
            tgt_x_max = min(tgt_image.shape[1], tgt_x_min + tgt_width + right_pad)
            tgt_y_max = min(tgt_image.shape[0], tgt_y_min + tgt_height + bottom_pad)

            # Resize the source ROI to fit the target bounding box
            resized_src_roi = cv2.resize(src_roi, (tgt_x_max - tgt_x_min, tgt_y_max - tgt_y_min))
            # cv2.imwrite(os.path.join(debug_folder, f"D_1_resized_src_roi_{file_name}"), (resized_src_roi * 255).astype(np.uint8))

            if visualize_once and not visualized:
            # Preview the source roi
                #plt.imshow(src_roi, cmap='gray')  # 'gray' colormap for single-channel images
                #plt.title("Source ROI")
                #plt.axis("off")  
                #plt.show()

                # Correct colour space 
                src_roi_rgb = cv2.cvtColor(src_roi, cv2.COLOR_BGR2RGB)
                plt.imshow(src_roi_rgb)
                plt.title("Source ROI")
                plt.axis("off")
                plt.show()
                 
                # Preview the resized roi
                resized_src_roi_rgb = cv2.cvtColor(resized_src_roi, cv2.COLOR_BGR2RGB)
                #plt.imshow(resized_src_roi, cmap='gray') 
                plt.imshow(resized_src_roi_rgb)
                plt.title("Resized source ROI to fit target bounding box")
                plt.axis("off")  
                plt.show()
                
            # Create a binary mask for the flag
            gray_roi = cv2.cvtColor(resized_src_roi, cv2.COLOR_BGR2GRAY)
            _, binary_mask = cv2.threshold(gray_roi, threshold, 1, cv2.THRESH_BINARY)

            # Dynamic thresholding using Otsu' method - red part of flag was retained 
            #_, binary_mask = cv2.threshold(gray_roi, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            
            # Note: re dynamic threhsold with Otsu's method above
            #       Otsu's method dynamically calculates a global threshold based on the image histogram. 
            #       When applied to a Red/Yellow flag, it obliterates the yellow part of the flag; the thresholding likely does not adequately distinguish 
            #       between the yellow flag and the background in grayscale. 
            
            binary_mask = binary_mask.astype(np.float32)

            #  Smooth the binary mask using Gaussian blur
            #  Kernel size (5, 5) controls the extent of smoothing.
            #  sigmaX determines the standard deviation of the Gaussian kernel. Lower values retain more detail, higher values smooth more aggressively.
            #  Expected improvements: Smoother Transitions: The blurred mask ensures that the blending region doesn't have jagged or harsh edges.
            #  More Natural Look: Morphological operations remove artifacts and make the edges more consistent.

            binary_mask = cv2.GaussianBlur(binary_mask, (5, 5), sigmaX=1)

            # Compute the gradient magnitude
            #sobelx = cv2.Sobel(binary_mask, cv2.CV_64F, 1, 0, ksize=3)
            #sobely = cv2.Sobel(binary_mask, cv2.CV_64F, 0, 1, ksize=3)
            #gradient_magnitude = cv2.magnitude(sobelx, sobely)
            # Normalize the gradient image
            #gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
            # Threshold the gradient magnitude
            #_, binary_mask = cv2.threshold(gradient_magnitude, 50, 1, cv2.THRESH_BINARY)

            if visualize_once and not visualized:
                # Preview the smoothed binary mask
                plt.imshow(binary_mask, cmap='gray')  # 'gray' colormap for single-channel images
                plt.title("Smoothed binary mask")
                plt.axis("off")  
                plt.show()
            
            # Binary mask: Shows the flag pixels detected within the bounding box. 
            # Helps verify the threshold's effectiveness in capturing only the desired region.
            # cv2.imwrite(os.path.join(debug_folder, f"D_2_binary_mask_smoothed_{file_name}"), (binary_mask * 255).astype(np.uint8))

            # Invert the binary mask to focus on the external background
            # The flag region (identified as 1 in the mask) becomes 0, and the background becomes 1. 
            # Allows the blending logic to target the background for blending, while preserving the flag interior.
            # The flag interior remains untouched, as its corresponding pixels in the mask are set to 0. 
            # Highlights the external region around the flag for blending and ensures that the focus is outside the flag.

            binary_mask = 1 - binary_mask

            if visualize_once and not visualized:
                # Preview the inverted binary mask
                plt.imshow(binary_mask, cmap='gray')  
                plt.title("Inverted binary mask")
                plt.axis("off")  
                plt.show()

            # cv2.imwrite(os.path.join(debug_folder, f"D_3_inverted_mask_{file_name}"), (binary_mask * 255).astype(np.uint8))
              
            # Expand the inverted mask slightly for better edge blending
            # Mask Expansion: Dilation ensures that the edges of the background around the flag blend better with the target image.

            # Improvements to make: 
            # 1. Adjustable mask expansion - Allow a parameter to control the kernel size:
            #        kernel = np.ones((kernel_size, kernel_size), np.uint8)  # Pass kernel_size as an argument
            #        binary_mask = cv2.dilate(binary_mask, kernel, iterations=1)
    
            #kernel = np.ones((3, 3), np.uint8)  # Used for original image sets that worked out well 
            kernel = np.ones((1, 1), np.uint8)  # Adjust kernel size if needed
            #binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_CLOSE, kernel, iterations=4)  # Close small holes
            #binary_mask = cv2.morphologyEx(binary_mask, cv2.MORPH_OPEN, kernel, iterations=1)   # Remove noise
            binary_mask = cv2.dilate(binary_mask, kernel, iterations=1)

            # Dilated Mask: Displays the expanded blending region around the flag and ensures smooth transitions at the edges
            # cv2.imwrite(os.path.join(debug_folder, f"D_4_dilated_mask_{file_name}"), (binary_mask * 255).astype(np.uint8))
   
            if visualize_once and not visualized:
                # Preview the inverted binary mask
                plt.imshow(binary_mask, cmap='gray')  # 'gray' colormap for single-channel images
                plt.title("Dilated mask")
                plt.axis("off")  
                plt.show()

                # cv2.imwrite(os.path.join(debug_folder, f"D_5_blended_region_{file_name}"), (blended_region * 255).astype(np.uint8))
                #print(f"Resized Source ROI Shape: {resized_src_roi.shape}")
                #print(f"Resized Source ROI Pixel Range: {resized_src_roi.min()} to {resized_src_roi.max()}")
    
                #print(f"Binary Mask Shape: {binary_mask.shape}")
                #print(f"Binary Mask Pixel Range: {binary_mask.min()} to {binary_mask.max()}")
    
                #print(f"Blended Region Coordinates: ({tgt_x_min}, {tgt_y_min}) to ({tgt_x_max}, {tgt_y_max})")
                #print(f"Target Image Region Shape: {tgt_image[tgt_y_min:tgt_y_max, tgt_x_min:tgt_x_max].shape}")
    
                #print(f"Blended Region Data Type: {blended_region.dtype}")
                #print(f"Blended Region Pixel Range: {blended_region.min()} to {blended_region.max()}")
          

            # Blend the flag's background with the target image
            for c in range(3):
                tgt_image[tgt_y_min:tgt_y_max, tgt_x_min:tgt_x_max, c] = (
                    resized_src_roi[:, :, c] * binary_mask +
                    tgt_image[tgt_y_min:tgt_y_max, tgt_x_min:tgt_x_max, c] * (1 - binary_mask)
            )

            # Visualise the blended region - only used for visualusation 
            blended_region = tgt_image[tgt_y_min:tgt_y_max, tgt_x_min:tgt_x_max]

            if visualize_once and not visualized:
                # Preview the blended region 
                plt.imshow(cv2.cvtColor(blended_region, cv2.COLOR_BGR2RGB))
                plt.title("Blended Region")
                plt.axis("off")
                plt.show()

            visualized = True  # Ensure visualizations occur only once

            # Update annotations for the new overlay
            #new_annotation = [
            #    source_class_id,
            #    round(tgt_x_center / tgt_image.shape[1], 6),
            #    round(tgt_y_center / tgt_image.shape[0], 6),
            #    round(tgt_width / tgt_image.shape[1], 6),
            #    round(tgt_height / tgt_image.shape[0], 6),
            #]


            # Explicitly recalculate tgt_x_center and tgt_y_center as the midpoint of the bounding box to ensure consistency between the overlay and the annotation.
            new_annotation = [
                source_class_id,
                round((tgt_x_min + tgt_width // 2) / tgt_image.shape[1], 6),  # x_center with x-offset applied
                round((tgt_y_min + tgt_height // 2) / tgt_image.shape[0], 6), # y_center
                round(tgt_width / tgt_image.shape[1], 6),                     # width
                round(tgt_height / tgt_image.shape[0], 6),                    # height
            ]

            #print(f"Target Bounding Box (Pixels): x_min={tgt_x_min}, y_min={tgt_y_min}, x_max={tgt_x_max}, y_max={tgt_y_max}")
            #print(f"New Annotation (YOLO Format): {new_annotation}")
            #print(f"Target Image Dimensions: {tgt_image.shape[1]} x {tgt_image.shape[0]}")

            break

    # Remove the target class annotation
    tgt_annotations = [annotation for annotation in tgt_annotations if int(annotation[0]) != target_class]

    return tgt_image, new_annotation, tgt_annotations

    
def process_switch_class(src_folder, tgt_folder, merge_folder, target_class, source_class_id, scale_factor, x_offset, flip_src=False, flip_tgt=False):
    
    countprocessed=0
    
    if not os.path.exists(merge_folder):
        os.makedirs(merge_folder)

    for file_name in os.listdir(src_folder):
        if file_name.lower().endswith('.png'):
            src_image = cv2.imread(os.path.join(src_folder, file_name))
            if flip_src:
                src_image = cv2.flip(src_image, 1)
            src_annotation_path = os.path.join(src_folder, os.path.splitext(file_name)[0] + ".txt")

            if not os.path.exists(src_annotation_path):
                #print(f"No annotations for {file_name} in source ({src_folder}). Skipping.")
                continue

            with open(src_annotation_path, 'r') as f:
                src_annotations = [list(map(float, line.split())) for line in f]

            tgt_image_path = os.path.join(tgt_folder, file_name)
            tgt_image = cv2.imread(tgt_image_path)
            if flip_tgt:
                tgt_image = cv2.flip(tgt_image, 1)
            tgt_annotation_path = os.path.join(tgt_folder, os.path.splitext(file_name)[0] + ".txt")

            if not os.path.exists(tgt_annotation_path) or tgt_image is None:
                #print(f"No target annotations or image for {file_name} in {tgt_folder}. Skipping.")
                continue

            with open(tgt_annotation_path, 'r') as f:
                tgt_annotations = [list(map(float, line.split())) for line in f]

            countprocessed = countprocessed +1
            # Save the first original target image for visualization
            if countprocessed == 1:
                cv2.imwrite(os.path.join(merge_folder, f"A_Source_image_{file_name}"), src_image)
                cv2.imwrite(os.path.join(merge_folder, f"B_Original_target_{file_name}"), tgt_image)

            # Fill target bounding box with background
            # background_filled_image = fill_target_bbox_with_background(tgt_image, tgt_annotations, target_class)
            # Selective padding - send 0 for the left or right depending on the relative position of the flagpole - could parameterize 
            background_filled_image = fill_target_bbox_with_background(
                tgt_image=tgt_image,
                tgt_annotations=tgt_annotations,
                target_class=target_class,
                padding=padding,  
                x_offset=x_offset
            )
            
            # Save the first background-filled target image for visualization
            # if countprocessed == 1:
            cv2.imwrite(os.path.join(merge_folder, f"C_Target_Image_with_flag_filled_{file_name}"), background_filled_image)

            # Overlay the source class onto the target class
            # updated_image, new_annotation, updated_annotations = overlay_class_region(
            #   src_image, src_annotations, source_class_id, background_filled_image, tgt_annotations, target_class, scale_factor, x_offset, edge_blend_width=30, debug_folder=merge_folder)

            updated_image, new_annotation, updated_annotations = overlay_class_region(
                file_name,
                src_image=src_image,
                src_annotations=src_annotations,
                source_class_id=source_class_id,
                tgt_image=background_filled_image,
                tgt_annotations=tgt_annotations,
                target_class=target_class,
                scale_factor = scale_factor,      
                x_offset = x_offset,      
                edge_blend_width = edge_blend_width,
                threshold = threshold,    
                align=align,  
                padding=padding,
                debug_folder=merge_folder
            )

            # Overlay the source class onto the target class
            #updated_image, new_annotation, updated_annotations = overlay_class_region_with_corrected_blending(
            #    src_image, src_annotations, source_class_id, background_filled_image, tgt_annotations, target_class, scale_factor, x_offset, edge_blend_width=10)

            if countprocessed == 1:
                # Save the first overlay image for visualization
                cv2.imwrite(os.path.join(merge_folder, f"E_overlay_result_{file_name}"), updated_image)

            # Create a new filename and save the updated image
            new_filename = create_merged_filename(file_name, merge_from_job, source_class_id, merge_to_job, target_class)
            output_path = os.path.join(merge_folder, new_filename)
            cv2.imwrite(output_path, updated_image)

            if countprocessed == 1:
                # visualize_pipeline_steps(target_image=tgt_image,background_filled=background_filled_image,resized_source=resized_src_roi,final_image=updated_image,file_name=file_name)
                # visualize_pipeline_steps(source_image=src_image,target_image=tgt_image,background_filled=background_filled_image, file_name=file_name) 
                visualize_pipeline_steps(source_image=f"A_Source_image_{file_name}",
                                             target_image=f"B_Original_target_{file_name}",
                                             background_filled=f"C_Target_Image_with_flag_filled_{file_name}", 
                                             final_image = f"E_overlay_result_{file_name}",
                                             file_name=file_name) 
           
            # Save the updated annotation file
            new_annotation_path = os.path.splitext(output_path)[0] + ".txt"
            with open(new_annotation_path, 'w') as f:
                for annotation in updated_annotations:
                    formatted_annotation = [int(annotation[0])] + [f"{coord:.6f}" for coord in annotation[1:]]
                    f.write(' '.join(map(str, formatted_annotation)) + '\n')
                if new_annotation:
                    formatted_annotation = [int(new_annotation[0])] + [f"{coord:.6f}" for coord in new_annotation[1:]]
                    f.write(' '.join(map(str, formatted_annotation)) + '\n')

#merge_class_from = "D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/Switch_flag_out_of_43_illum"
#merge_class_into = "D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/Switch_flag_into_88"
#merge_fldr = "D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/Switch_class_1_from_43_into_88/Merge_43_to_88_with_fillC"

merge_class_from = 'D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/input'
merge_class_into = 'D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/target'
merge_fldr = 'D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/target32'
merge_from_job = '30' # Just used in naming 
merge_to_job = '24'   # Just used in naming 

class_to_take = 1     # Source class to be merged over the target class in the new image 
class_to_replace = 0  # Target class to be replaced 
scale_factor = 1.2    # Scaling factor for the source region
x_offset = 20 #-80 #20         # Horizontal offset for overlay placement - fine tune differently 
edge_blend_width = 20  # Width of edge blending
threshold = 190 # 190 #245       # 190 Intensity threshold for binary mask  
debug_folder = merge_fldr  # Path to save debugging visualizations
align = 'top-left'  #'top-left' # Alignment option ("top-left" or "top-right")
padding=(5, 5, 5, 0)   # Padding: (top, right, bottom, left) #padding=(0, 5, 5, 5)
#border_size=10,      # Sample colour regions 

# Ensure debug folder exists
#if not os.path.exists(debug_folder):
#    os.makedirs(debug_folder)

process_switch_class(merge_class_from, merge_class_into, merge_fldr, class_to_replace, class_to_take, scale_factor, x_offset, flip_src=False, flip_tgt=False)


Visualization saved at: D:/FlagDetectionDatasets/Augmentation/Switch_class_in_images_sets/target32\F_pipeline_steps_000328.PNG
